In [23]:
import sys

import pandas as pd
import tensorflow as tf

from collections import Counter

In [2]:
sys.path.append('..')

In [21]:
from src.preprocessing.tokens import tokenize_method, split_subtokens
from src.utils.pandas import lists_to_series

In [7]:
!wc -l ../data/interim/repositories/all.csv

792035 ../data/interim/repositories/all.csv


In [19]:
csv_filename = '../data/interim/repositories/all.csv'

max_input_vocab_size=30000

In [11]:
# Reading the input files
df = pd.read_csv(csv_filename)

df.head()

,file_name,class_name,method_name,return_type,arguments,body,documentation,modifiers,annotations
0,../external/repositories/antlr4/doc/resources/...,CaseChangingCharStream,getText,String,"[['interval', 'Interval']]",{ return stream.getText(interval); },None,['public'],['Override']
1,../external/repositories/antlr4/doc/resources/...,CaseChangingCharStream,consume,void,[],{ stream.consume(); },None,['public'],['Override']
2,../external/repositories/antlr4/doc/resources/...,CaseChangingCharStream,LA,int,"[['i', 'int']]",{ int c = stream.LA(i); if (c <= 0) { return c...,None,['public'],['Override']
3,../external/repositories/antlr4/doc/resources/...,CaseChangingCharStream,mark,int,[],{ return stream.mark(); },None,['public'],['Override']
4,../external/repositories/antlr4/doc/resources/...,CaseChangingCharStream,release,void,"[['marker', 'int']]",{ stream.release(marker); },None,['public'],['Override']


In [12]:
# Cleaning, filtering the data
df = df.dropna()
len(df)

791986

In [13]:
# Tokenize and filter input sequences
df['inputs'] = df['body'].progress_apply(tokenize_method)

 14%|█▍        | 114426/791986 [00:51<03:57, 2851.85it/s]

ERROR in tokenizing: {packagesToScan}") String... packagesToScan) return new ServiceClassPostProcessor(packagesToScan); }


 58%|█████▊    | 458503/791986 [03:49<02:08, 2594.95it/s]

ERROR in tokenizing: {logFile : .+}") public void parseFile( @PathParam("logFile") String logFile) throws IOException, SkylineStoreException, ResourceEstimatorException { logParserUtil.parseLog(logFile); LOGGER.debug("Parse logFile: {}.", logFile); }


 93%|█████████▎| 737234/791986 [05:56<00:37, 1443.33it/s]

...

 97%|█████████▋| 771638/791986 [06:12<00:06, 3078.40it/s]

ERROR in tokenizing: {2 == (1+1)}") Boolean spelParameterValue) this.spelParameterValue = spelParameterValue; }


 98%|█████████▊| 776106/791986 [06:14<00:09, 1608.72it/s]

ERROR in tokenizing: {myKey}") String id, @RequestHeader(defaultValue = "#{systemProperties.myHeader}") String header, @Value("#{request.contextPath}") String contextPath, HttpServletResponse response) throws IOException { response.getWriter().write(String.valueOf(id) + "-" + String.valueOf(header) + "-" + contextPath); }


100%|██████████| 791986/791986 [06:23<00:00, 2063.19it/s]


In [14]:
# Tokenize output sequences
df['outputs'] = df['method_name'].progress_apply(split_subtokens)

100%|██████████| 791986/791986 [00:05<00:00, 136357.69it/s]


In [15]:
# Filter out the samples which cannot be tokenized
df = df[df.inputs.str.len() > 0]

In [20]:
df['inputs'].head()

0    [{, return, stream, ., get, Text, (, interval,...
1                  [{, stream, ., consume, (, ), ;, }]
2    [{, int, c, =, stream, ., LA, (, i, ), ;, if, ...
3             [{, return, stream, ., mark, (, ), ;, }]
4          [{, stream, ., release, (, marker, ), ;, }]
Name: inputs, dtype: object

In [24]:
unique_body_tokens = pd.DataFrame.from_dict(Counter(lists_to_series(df['inputs']).values), orient='index') \
    .sort_values(by=0, ascending=False) \
    .reset_index().rename(columns={'index': 'token', 0: 'count'})

unique_body_tokens.head(30)

,token,count
0,),4494423
1,(,4457284
2,.,3179357
3,;,2645624
4,",",1835914
5,{,1480658
6,},1221188
7,=,904555
8,<STR>,825293
9,get,660809


In [31]:
unique_body_tokens[unique_body_tokens['count'] >= 5]

,token,count
0,),4494423
1,(,4457284
2,.,3179357
3,;,2645624
4,",",1835914
...,...,...
35246,_ROTATE_LEFT,5
35247,EXTLEN,5
35248,similarities,5
35249,_PER_DEGREES,5


In [34]:
unique_body_tokens[unique_body_tokens.token.str.contains('^[0-9]+$')]

,token,count
16,0,258968
18,1,238329
26,2,137882
77,3,55054
139,4,29792
...,...,...
102028,10011011110,1
102029,10011011101,1
102049,12111,1
102050,4502,1


In [17]:
def get_vocab_index(df, max_vocab_size):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=max_vocab_size,
        filters='',
        lower=False,
        oov_token='<OOV>',
    )

    tokenizer.fit_on_texts(df.values)

    return tokenizer

In [ ]:
input_tokenizer = get_vocab_index(
    df['inputs'],
    max_vocab_size=max_input_vocab_size
)
input_vocab_index = input_tokenizer.word_index
